In [1]:
import numpy as np

# Create a Generator using MT19937
bitgen = np.random.MT19937(seed=42)
rng = np.random.Generator(bitgen)

# Use this generator to sample
print(rng.uniform(0, 1, 1))  # Now uses seeded MT19937

[0.54199389]


In [2]:
#import numpy as np
import random
import heapq
from scipy.stats import t
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import copy


Considering that **Ward F is allocated 34 beds**, the **total number of beds** available for the remaining wards is:

$$
B_N = 165 \quad \Rightarrow \quad B_{\text{remaining}} = B_N - 34 = 131
$$

To estimate how many beds should be allocated to **Ward A**, we compute a **priority-weighted share** based on:

- $ \lambda_i $: patient arrival rate to ward $ i $
- $ \mu_i $: service rate (i.e., discharge rate), where $ \frac{1}{\mu_i} $ is the mean length of stay
- $ UP_i $: urgency points for ward $ i $

We define a **priority-weighted blocking likelihood** for Ward A as:

$$
\Pr(B_A) = \frac{\lambda_A \cdot \frac{1}{\mu_A} \cdot UP_A}{\sum\limits_{i \in \{A, B, C, D, E\}} \lambda_i \cdot \frac{1}{\mu_i} \cdot UP_i}
$$

Then, the **suggested number of beds** to allocate to Ward A is:

$$
B_A = \Pr(B_A) \cdot B_{\text{remaining}} = \Pr(B_A) \cdot (B_N - 34)
$$

This allocation accounts for each ward's demand pressure (arrival rate and length of stay) and the criticality of avoiding rejections (urgency points).


In [3]:
# Define parameters for wards A–E
wards = ['A', 'B', 'C', 'D', 'E']

# Given data
arrival_rates = {'A': 14.5, 'B': 11.0, 'C': 8.0, 'D': 6.5, 'E': 5.0}
mean_stays = {'A': 2.9, 'B': 4.0, 'C': 4.5, 'D': 1.4, 'E': 3.9}  # 1/mu
urgency_points = {'A': 7, 'B': 5, 'C': 2, 'D': 10, 'E': 5}

# Total beds and fixed beds for Ward F
total_beds = 165
beds_for_F = 34
remaining_beds = total_beds - beds_for_F

# Calculate priority weights
priority_weights = {}
denominator = sum(arrival_rates[w] * mean_stays[w] * urgency_points[w] for w in wards)
for w in wards:
    numerator = arrival_rates[w] * mean_stays[w] * urgency_points[w]
    priority_weights[w] = numerator / denominator

# Compute bed allocation
bed_allocation = {w: round(priority_weights[w] * remaining_beds) for w in wards}
import pandas as pd

# Reconstruct bed allocation dictionary as a DataFrame
bed_allocation_df = pd.DataFrame.from_dict(bed_allocation, orient='index', columns=["Estimated Beds"])
bed_allocation_df.index.name = "Ward"
bed_allocation_df.reset_index(inplace=True)

bed_allocation_df

,Ward,Estimated Beds
0,A,50
1,B,37
2,C,12
3,D,15
4,E,16
